# Data Science Jobs on Glassdoor

### The main goal of this study is to compare the differing salaries between different sectors for Data Professionals.

## Connecting to Data Source

In [36]:
import sqlite3 as sql

db = sql.connect('dsjobs')
c = db.cursor()

c.execute('select * from jobs')
mydata = list(c.fetchall())

In [37]:
import pandas as pd

data = pd.DataFrame(mydata)
data.columns = ['Index', 'JobTitle', 'SalaryEstimate', 'JobDescription', 'Rating',
              'CompanyName', 'Location', 'Headquarters', 'Size', 'Founded',
              'TypeOfOwnership', 'Industry', 'Sector', 'Revenue', 'Competitors']
data = data.set_index('Index')
data.head()

,JobTitle,SalaryEstimate,JobDescription,Rating,CompanyName,Location,Headquarters,Size,Founded,TypeOfOwnership,Industry,Sector,Revenue,Competitors
Index,,,,,,,,,,,,,,
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## Data Exploration + Some Basic Data Manipulation

In [61]:
df = data.copy()

df['CompanyName'] = df['CompanyName'].apply(lambda x: x[:-4])
print(df.CompanyName[:5])

df['JobDescription'] = df['JobDescription'].apply(lambda x: x.replace('\n', ' '))
print(df['JobDescription'][:5])

Index
0           Healthfirst
1               ManTech
2        Analysis Group
3               INFICON
4    Affinity Solutions
Name: CompanyName, dtype: object
Index
0    Description  The Senior Data Scientist is resp...
1    Secure our Nation, Ignite your Future  Join th...
2    Overview   Analysis Group is one of the larges...
3    JOB DESCRIPTION:  Do you have a passion for Da...
4    Data Scientist Affinity Solutions / Marketing ...
Name: JobDescription, dtype: object


In [46]:
df['JobTitle'].value_counts(ascending=False)

Data Scientist                                            337
Data Engineer                                              26
Senior Data Scientist                                      19
Machine Learning Engineer                                  16
Data Analyst                                               12
                                                         ... 
Data Science Instructor                                     1
Business Data Analyst                                       1
Purification Scientist                                      1
Data Engineer, Enterprise Analytics                         1
AI/ML - Machine Learning Scientist, Siri Understanding      1
Name: JobTitle, Length: 172, dtype: int64

In [62]:
df['SalaryEstimate'] = df['SalaryEstimate'].apply(lambda x: x.split('(')[0])
df['SalaryEstimate'] = df['SalaryEstimate'].apply(lambda x: x.replace('K', '000'))
df['SalaryEstimate'] = df['SalaryEstimate'].apply(lambda x: x.replace('$', ''))
df['SalaryEstimate'] = df['SalaryEstimate'].apply(lambda x: x.split('-'))

In [58]:
list(map(int, ['1', '2']))

[1, 2]

In [72]:
import numpy as np

def get_stats(column):
    functions = [np.min, np.max, np.mean, np.median]
    col_names = [column + 'Min', column + 'Max', column + 'Mean', column + 'Median']

    for k, v in enumerate(functions):
        df[col_names[k]] = df[column].apply(lambda x: v(x))

In [74]:

df['SalaryEstimate'] = df['SalaryEstimate'].apply(lambda x: list(map(int, x)))
get_stats('SalaryEstimate')

df[['SalaryEstimateMin', 'SalaryEstimateMax', 'SalaryEstimateMean', 'SalaryEstimateMedian']][:5]

,SalaryEstimateMin,SalaryEstimateMax,SalaryEstimateMean,SalaryEstimateMedian
Index,,,,
0,137000,171000,154000.0,154000.0
1,137000,171000,154000.0,154000.0
2,137000,171000,154000.0,154000.0
3,137000,171000,154000.0,154000.0
4,137000,171000,154000.0,154000.0


## Statistical Analysis

### T-Test

Our statistical question is to see if it matters which work sector a data professional works in, in regards to their pay. This is an important question for all data professionals. 

#### Methodology
- For this test I will be running a one sample t-test for each of the sectors comparing it to the average for all Data Professionals.
- To avoid some bias I will use just Data Scientist job titles.

In [76]:
columns = ['Sector', 'Industry', 'SalaryEstimateMean', 'SalaryEstimateMedian', 'SalaryEstimateMin', 
       'SalaryEstimateMax']

test_data = df[columns].copy()

In [91]:
grouped = test_data.groupby('Sector')[columns[2:]]
sectors = test_data['Sector'].unique()

In [104]:
grouped.get_group('Non-Profit').SalaryEstimateMean

Index
477    43500.0
Name: SalaryEstimateMean, dtype: float64

In [145]:
def create_dict(category, grouped=grouped, sectors=sectors):
    grouped_data = dict()
    for sector in sectors:
        grouped_data[sector] = grouped.get_group(sector)[category]

    return grouped_data

average_pay = create_dict(category='SalaryEstimateMean')

In [149]:
from scipy.stats import ttest_1samp

average_pay_total = np.mean(test_data['SalaryEstimateMean'])

def run_ttest(average_pay, average_pay_total):
    ttest_results = dict()

    for k, v in average_pay.items():
        ttest_results[k] = ttest_1samp(average_pay[k], average_pay_total, alternative='less').pvalue

    return ttest_results

ttest = run_ttest(average_pay, average_pay_total)
del ttest['Non-Profit']

### Below are the results from first T-Test

In [147]:
ttest

{'Insurance': 0.008081499778247766,
 'Business Services': 0.9329296719364715,
 'Manufacturing': 0.47679165865610074,
 'Information Technology': 0.026006219578342,
 'Biotech & Pharmaceuticals': 0.44127691998062263,
 'Retail': 0.9538666866344652,
 'Oil, Gas, Energy & Utilities': 0.020210486387884957,
 'Government': 0.9091629788290714,
 'Health Care': 0.2657839149292657,
 'Finance': 0.0810316986266039,
 'Aerospace & Defense': 0.9023822767667143,
 '-1': 0.8858670254655299,
 'Transportation & Logistics': 0.2672663583279519,
 'Media': 0.8063823852752741,
 'Telecommunications': 0.0584286234486969,
 'Real Estate': 0.29191826220652595,
 'Travel & Tourism': 0.5624348153295925,
 'Agriculture & Forestry': 0.04050673250725828,
 'Education': 0.37670644556064103,
 'Accounting & Legal': 0.5195022603441198,
 'Construction, Repair & Maintenance': 0.1809650738243079,
 'Consumer Services': 0.7765002075759576}

### These are the results of a one-sided, for lower, T-Test. This tested to see if any of the Sectors had a lower salaries than average. This could important for aspiring data professionals to know before apply to certain companies. Below are some graphics to make this more clear. 

In [181]:
import plotly.express as px

fig = px.bar(x=ttest.keys(), y=ttest.values(), title='P-Values of Different Sectors')
fig.show()

In [151]:
less_than_alpha = {k: v for k, v in ttest.items() if v < 0.05}
less_than_alpha

{'Insurance': 0.008081499778247766,
 'Information Technology': 0.026006219578342,
 'Oil, Gas, Energy & Utilities': 0.020210486387884957,
 'Agriculture & Forestry': 0.04050673250725828}

In [152]:
less_than_alpha_fig = px.bar(
    x=less_than_alpha.keys(), y=less_than_alpha.values(),
    title='Sectors with Bad Pay'
)
less_than_alpha_fig.show()

### In conclusion, the 4 sectors where the null hypothesis was rejected, and there was staistical signifigance to show that mean pay was lower than the average of all sectors was: Insurance, Information Technology, (Oil, Gas, Energy & Utility's).

### I will run the same type of ttest for Minimum and Maximum of the ranges for pay

## Minimum Pay TTest

In [191]:
minimum_pay = create_dict(category='SalaryEstimateMin')
minimum_pay_total = np.mean(test_data['SalaryEstimateMin'])
ttest_min = run_ttest(minimum_pay, minimum_pay_total)
ttest_min

{'Insurance': 0.014925122494572483,
 'Business Services': 0.9413808310149521,
 'Manufacturing': 0.6024782346282191,
 'Information Technology': 0.025074161295245864,
 'Biotech & Pharmaceuticals': 0.38744111450801877,
 'Retail': 0.9586481327700485,
 'Oil, Gas, Energy & Utilities': 0.05537756168549447,
 'Government': 0.8926854164562975,
 'Health Care': 0.3714574793737172,
 'Finance': 0.08035147469607919,
 'Aerospace & Defense': 0.9371341853727252,
 '-1': 0.8850634741859565,
 'Transportation & Logistics': 0.1924207092792538,
 'Media': 0.8452732521584114,
 'Telecommunications': 0.05874984921970325,
 'Real Estate': 0.16059031160727483,
 'Travel & Tourism': 0.6125918326782319,
 'Agriculture & Forestry': 0.009742115752979114,
 'Education': 0.4370605100997359,
 'Accounting & Legal': 0.5128694450247087,
 'Non-Profit': nan,
 'Construction, Repair & Maintenance': 0.16265979608258907,
 'Consumer Services': 0.7663621310070343}

In [180]:
ttest_min_fig = px.bar(x=ttest_min.keys(), y=ttest_min.values(), title='P-Values of Minimum Salary TTest')
ttest_min_fig.show()

In [171]:
del ttest_min['Non-Profit']

In [172]:
min_less_than_alpha = {k: v for k, v in ttest_min.items() if v < 0.05}
min_less_than_alpha

{'Insurance': 0.014925122494572483,
 'Information Technology': 0.025074161295245864,
 'Agriculture & Forestry': 0.009742115752979114}

In [177]:
min_less_than_alpha_fig = px.bar(
    x=min_less_than_alpha.keys(), y=min_less_than_alpha.values(),
    title='Minimum Salaries Below the Minimum Average'
)
min_less_than_alpha_fig.show()

### This chart is showing us something important! So far these three sectors are repeats from average salary as well. Lets see what the maximum salary looks like.

## Maximum Pay TTest

In [189]:
maximum_pay = create_dict(category='SalaryEstimateMax')
maximum_pay_total = np.mean(test_data['SalaryEstimateMax'])
ttest_max = run_ttest(maximum_pay, maximum_pay_total)
ttest_max

{'Insurance': 0.00789547620788003,
 'Business Services': 0.9209614054702427,
 'Manufacturing': 0.37713157227475047,
 'Information Technology': 0.033413989100626394,
 'Biotech & Pharmaceuticals': 0.47917981467846676,
 'Retail': 0.9451362135552798,
 'Oil, Gas, Energy & Utilities': 0.011215884590353067,
 'Government': 0.9048636891183643,
 'Health Care': 0.2102161388223649,
 'Finance': 0.09315102823014665,
 'Aerospace & Defense': 0.8612148390211138,
 '-1': 0.8786740741777784,
 'Transportation & Logistics': 0.3378897475892124,
 'Media': 0.780745405740022,
 'Telecommunications': 0.08435633381747197,
 'Real Estate': 0.4391445477770951,
 'Travel & Tourism': 0.5273607778674436,
 'Agriculture & Forestry': 0.10685404252332516,
 'Education': 0.29920777372042573,
 'Accounting & Legal': 0.5221792470885993,
 'Non-Profit': nan,
 'Construction, Repair & Maintenance': 0.1948272538406659,
 'Consumer Services': 0.7827284686805097}

In [178]:
del ttest_max['Non-Profit']

In [182]:
ttest_max_fig = px.bar(x=ttest_max.keys(), y=ttest_max.values(), title='Maximum Salary P-Values')
ttest_max_fig.show()

In [185]:
max_less_than_alpha = {k: v for k, v in ttest_max.items() if v < 0.05}
max_less_than_alpha

{'Insurance': 0.00789547620788003,
 'Information Technology': 0.033413989100626394,
 'Oil, Gas, Energy & Utilities': 0.011215884590353067}

In [187]:
max_less_than_alpha_fig = px.bar(
    x=max_less_than_alpha.keys(), y=max_less_than_alpha.values(),
    title='Sectors with Less than Average Maximum Salaries'
)
max_less_than_alpha_fig.show()

#### Through hypothesis testing, there is statistical evidence on all 3 ttest's that Insurance and Information Technology has worse than average salaries for Minimum Salary, Average Salary, and Maximum Salary. This said Insurance and Information Technology are the two worst sectors to work in as a data professional in regards to pay. Of course, there are many factors wehn considering a job. This study just focusses on the pay aspect.